#### 사용자 정의 데이터셋과 모델
 - iris.csv ==> 사용자 정의 데이터셋
 - DNN 모델 ==> 사용자 정의 모델

[1] 모듈로딩 <hr>

In [1]:
# 모듈 로딩

import torch                                    # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 손실, 거리 등 함수 관련 모듈
import torch.optim as optim                     # 최적화 기법 관련 모듈
from torchmetrics.classification import F1Score # 성능 지표 관련 모듈
from torchinfo import summary                   # 모델 정보 관련 모듈

from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd                 # 데이터 파일 분석 관련 모듈

In [2]:
# 데이터
DATA_FILE = '../data/iris.csv'

In [3]:
# CSV => DF
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [4]:
# 타겟 컬럼 수치화 ==>  LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성
<hr>

In [5]:
#-------------------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : CustomDataSet
# 부모 클래스 : torch.utils.data.dataset
# 매개 변수 : featureDF, targetDF
# >> 밖에서 전처리한걸 들고왔음
#-------------------------------------------------------


class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,featureDF, targetDF) :
        super().__init__()
        self.featureDF = featureDF 
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 변환!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS
        

[3] 데이터셋 인스턴스 생성 <hr>

In [6]:
# 피쳐와 타게 추출
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]  # irisDF[[irisDF.columns[-1]]]도 가능
print(f'featureDF => {featureDF.shape} , targetDF => {targetDF.shape}')

featureDF => (150, 4) , targetDF => (150, 1)


In [7]:
# iris 데이터셋 인스턴스 생성
irisDS = CustomDataset(featureDF, targetDF)

In [8]:
# iris 데이터셋 속성 >> [상자] 모양
irisDS.featureDF , irisDS.n_features, irisDS.n_rows

(     sepal.length  sepal.width  petal.length  petal.width
 0             5.1          3.5           1.4          0.2
 1             4.9          3.0           1.4          0.2
 2             4.7          3.2           1.3          0.2
 3             4.6          3.1           1.5          0.2
 4             5.0          3.6           1.4          0.2
 ..            ...          ...           ...          ...
 145           6.7          3.0           5.2          2.3
 146           6.3          2.5           5.0          1.9
 147           6.5          3.0           5.2          2.0
 148           6.2          3.4           5.4          2.3
 149           5.9          3.0           5.1          1.8
 
 [150 rows x 4 columns],
 4,
 150)

In [9]:
# iris 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] DataLoader 인스턴스 생성 <hr>

In [10]:
## 필요한 것 : Dataset 인스턴스, Batch_size
irisDL = DataLoader(irisDS, batch_size=10)

In [11]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([10, 4]) torch.Size([10, 1])


[5] 모델 준비 <hr>


In [12]:
#-------------------------------------------------------
# 모델 이름 : CustomModel
# 부모클래스 : nn.Module
# 매개변수 : None 
# 모델 구조
# - 입력층 : 입력  4개   출력 10개   AF - ReLU -> LeakyReLU
# - 입력층 : 입력 10개   출력 30개   AF - ReLU -> LeakyReLU
# - 입력층 : 입력 30개   출력  3개(ovr)   AF >> 분류 - 다중 softmax
#-------------------------------------------------------

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_layer = nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,3)

    # 순방향 학습 메서드
    def forward(self, x):
        y= F.relu(self.in_layer(x))
        y= F.relu(self.hidden_layer(y))
        return self.out_layer(y)

In [13]:
### 모델 인스턴스 생성
model = CustomModel()
print(model)
summary(model)

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            50
├─Linear: 1-2                            330
├─Linear: 1-3                            93
Total params: 473
Trainable params: 473
Non-trainable params: 0

In [14]:
summary(model, input_size=(1000,4))

Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              [1000, 3]                 --
├─Linear: 1-1                            [1000, 10]                50
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 3]                 93
Total params: 473
Trainable params: 473
Non-trainable params: 0
Total mult-adds (M): 0.47
Input size (MB): 0.02
Forward/backward pass size (MB): 0.34
Params size (MB): 0.00
Estimated Total Size (MB): 0.36

In [28]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

[6] 학습 <hr>

In [30]:
## 배치크기만큼 데이터와 타겟 추출해서 학습 진행 >>>????
TS_loss, TS_score = [[],[]] , [[],[]]
for epoch in range(10):
    loss_total, score_loss = 0, 0 
    for dataTS, targetTS in irisDL:

        # 배치크기 만큼의 학습 데이터
        # print(dataTS.shape, targetTS.shape, targetTS.dtype)  # >> 확인용
        targetTS = targetTS.reshape(-1).long()
    
        # 배치크기 만큼 학습 진행
        pre_y = model(dataTS)
        # print(pre_y.shape, targetTS.reshape(-1).shape) # 확인결과 >> shape이 맞지않음 형태 맞춤

        # 손실 계산 >> 이진분류(BCELoss)랑 다중분류(CrossEntropyLoss)랑 나눠 놨음
        # 주의 다중분류(CrossEntropyLoss)에서는 softmax가 포함 되어있다.
        loss= nn.CrossEntropyLoss()(pre_y,targetTS)
        loss_total += loss.item()

        # 최적화
        optimizer.zero_grad()
        optimizer.step()

        # 점수 추출
        score = F1Score(task='multiclass',num_classes=3)(pre_y,targetTS)
        score_loss += score.item()

    TS_loss[0].append(loss_total/len(irisDL))
    TS_score[0].append(score_loss/len(irisDL))

print(f'[{epoch}/10]\n -[Train] LOSS : {TS_loss[0]}\n  SCORE : {TS_score[0]}')


  

[9/10]
 -[Train] LOSS : [1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524, 1.2723413864771524]
  SCORE : [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [43]:
# argmax